In [1]:
# tramite API si può ottenere anche il weight del game, ormai l'ho già inserito nel dataset e va bene così 
#però ricordarsi che c'è questa opzione, c'è anche num weights.

Che manca di sicuro è:
number of choices, luck.
bookkeping è solo per una certa nicchia e non ne vale la pena perderci la testa, magari si può fare allenando un altro modello di AI o semplicemente prendendo clusters di articoli riguardanti la contabilità, clusterizzare il rulebook insieme ad essi o solo le pagine del rulebook relative alla contabilità e vedere quanto esse sono vicine a questi articoli di contabilità.
Se sono abbastanza vicine o hanno parole in comune con tfidf alto allora hanno una buona contabilità al loro interno quindi può venir calcolata così per similarità a questi articoli.

la luck dev'essere calcolata considerando dadi, carte facendo differenza tra scelte o pescaggio,
anche qua potrei prendere varie frasi di esempio dai vari rulebooks e utilizzarle per allenare un altro modello di AI che poi mi aiuti a capire quante frasi all'interno del rulebook sono relative alla randomness e  cercare di pesare il tutto correttamente.
oppure cercando pattern fissi es: lancio dadi, pescaggio carte.
rolls dice, quant'è il dado? quante carte hai da pescare? definiscono la luck
potrei approssimare a: pescare carte 1, roll dice 1, unexpected events?

technical skills è ben approssimato dai readability index

amount of choices available è molto difficile da considerare, tipo in GO la complessità è molto alta perchè le possibilità sono molto elevate.
per poter capire quante scelte fare dovrei conoscere quali sono le regole e basandomi su esse fare una specie di tree o grafo per capire quanti possibili path possono essere presi all'interno di un turno.
questo risulterebbe mooolto dispendioso, devo trovare un modo per approssimarlo oppure per evitarlo

## definition of complex
Definition of complex
1: a whole made up of complicated or interrelated parts.

2:A group of obviously related units of which the degree and nature of the relationship is imperfectly known

Sometime around 2017 it was decided not to display on a game's main page its average User Rating if the game has fewer than 30 ratings (since a small number of user ratings can be so extremely variable).

It was later decided to nonetheless show the average rating if the game had been in the database for over a year and was still struggling to get 30 ratings.



# (by BGG) Factors that affect a game's weight include:
### "Weight" is not actually defined at BGG so different people have different ideas of what it means. The choices for Game Play Weight Are:
#### Amount of rules     (contare cap sotto cap.) oppure approssimare con num sentences, visto che cap e sotto capitoli risulterebbe un po' troppo una stima al ribasso
#### Gameplay length    X
#### Amount of luck       (contare numero di elementi aleatori), I would say, more luck needed less skills is required by the players
#### Technical skill required (math, planning, reading, etc.)    X (readability index, higher the index, higher needs to be the technical skill of the player)
#### Amount of choices available (?)
#### Amount of bookkeeping (?), contabilità, (CONTARE CAPITORLI CON TOPIC = BOOKKEPING, fare modello per trovare testi riguardo bookkeping.)
#### Level of difficulty  (!), this feature is redundant, I need to understand the level of difficulty

# What I added to this BGG definition:
#### readability indexes also make you understand how much is actually difficult to understand the rules and game mechanics
#### avg play age
#### dependency on text while playing the game

In [2]:
#voglio un dataset del tipo: id (versione base), rulebook, weight, weight potrebbe essere estratto dalle API
#ma comunque va bene averlo già nel dataset

#siccome il weight non sembra esserci nelle API di bgg lo dovrò inserire a mano,
#dopo aver ottenuto queste caratteristiche a mano sarà possibile ottenerne altre sulla base delle BGG API
#le features da estrarre saranno la readability dal rulebook e altre features caratteristiche del game
#sarebbe utile anche capire la quantità di regole e meccanismi possibili dal rulebook

#il dataset è formato praticamente da boardgames solo su giochi di guerra, sarebbe comodo aggiungere
#giochi più fantasy

# TIP: ADD MORE GAMES, find another good feature like: number of rules by counting the numbers of chapters and sub chapters!!!!

# oppure provare con numero di componenti per giocare, tipo carte statuette ecc

## Creating Dataset

In [139]:
import pandas as pd
data = pd.read_csv('BoardGames.csv')
data # to display the first 5 lines of loaded data

,Name,Id,ruleBook,weight
0,Gloomhaven,174430,gloomhaven-rulebook.pdf,3.88
1,austerlitz-1805-napoleons-greatest-victory,8571,aus_rules.pdf,3.20
2,commands-colors-ancients,14105,CC-EPIC_Rules.pdf,2.69
3,conquerors-alexander-great,13224,conq_alex_Rules.pdf,2.84
4,dominant-species-card-game,96260,DSC_Rules-web_FINAL.pdf,1.91
...,...,...,...,...
69,marvel-champions-card-game,285774,c2-marvel-champions-the-card-game-rules-refere...,2.89
70,ark-nova,342942,Ark-Nova-Rulebook.pdf,3.71
71,brass-birmingham,224517,Brass-Birmingham-Rulebook-2018.11.20-highlight...,3.91
72,7-wonders-duel,173346,6b-7-wonders-duel-rules.pdf,2.22


In [4]:
import PyPDF2
from tqdm import tqdm 

rulebooks = {} #for each game a rulebook

dataStrip = data[['Name', 'ruleBook']]

#extracting text from pdfile:
for row in dataStrip.itertuples():
    rulebook = row[2]
    name = row[1]
    elToOpen = "dataset/" + rulebook
    pdfFileObj = open(elToOpen, 'rb')
 
    # creating a pdf reader object
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj) 
    text = []
    for i in tqdm(range(0, pdfReader.numPages)):
        pageObj = pdfReader.getPage(i)
        text.append(pageObj.extract_text()) #text = whole rulebook
    rulebooks[name] = text
    # closing the pdf file object
    pdfFileObj.close()

print(rulebooks['Gloomhaven'])

100%|██████████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<00:00, 49.18it/s]
C:\ProgramData\Anaconda3\lib\site-packages\PyPDF2\_reader.py:1319: PdfReadWarning: Xref table not zero-indexed. ID numbers for objects will be corrected.
  PdfReadWarning,
100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:01<00:00,  6.36it/s]

[' STOP! If you would prefer to watch a video \nexplanation of the rules, please visit \n\nwww.cephalofair.com/gaming-rules-overview', '1\n Rule Book\n\n\n\n\n18\n\n Character Miniatures\n\n\n\n\n47\n\n Monster Stat Sheets\n\n\n\n\n24\n\n Battle Goal Cards\n\n\n\n\n50\n\n Money Tokens\n\n\n\n\n\n1\n\n Scenario Book\n\n\n\n\n17\n\n Character Boards\n\n\n\n\n\n6\n Monster Stat Sleeves\n\n\n\n\n24\n\n Personal Quest Cards\n\n\n\n\n\n46\n\n Damage Tokens\n\n\n\n\n\n1\n\n Town Records Book\n\n\n\n\n504\n\n Character Ability Cards\n\n\n\n\n\n150\n\n Event Cards\n\n\n\n\n9\n\n Random Scenario Cards\n\n\n\n\n10\n\n Scenario Aid Tokens\n\n\n\n\n\n1\n\n Map Board\n\n\n\n\n457\n\n Attack Modifier Cards\n\n\n\n\n\n253\n\n Item Cards\n\n\n\n\n40\n\n Random Dungeon Cards\n\n\n\n\n4\n\n HP/XP Dials\n\n\n\n\n\n12\n\n Objective Tokens\n\n\n\n\n\n4\n\n Player Reference Cards\n\n\n\n\n\n30\n\n 2-Sided Map Tiles\n\n\n\n\n240\n\n Monster Standees\n\n\n\n\n\n6\n\n Wood Element Discs\n\n\n\n32 \n\nSummon Tok

In [5]:
example = rulebooks['7-wonders-duel']

In [6]:
#weights van presi con API, almeno provarci, se no riempio il dataset a mano. perchè magari non matchano i nomi

In [7]:
#1st step:
#provare a calcolare il numero di sillabe medio, vedere se eliminare junks o cosa.

In [8]:
import spacy
from tqdm import tqdm 
from nltk.tokenize import word_tokenize
import string
import re

In [9]:
#guardare codice trovato online, estrarre features, provare a creare il dataset con le rulebooks e labels i weights,
#per ogni rulebook estrarre le features e provare a creare un regression model per predirre il weight

In [10]:
#Splits the text into sentences
def break_sentences(text):
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)
    return list(doc.sents)



#Returns the number of sentences
def sentence_count(text):
    return len(break_sentences(text))

#Returns average sentence length
def avg_sentence_length(text):
    words = word_count(text)
    sentences = sentence_count(text)
    average_sentence_length = float(words / sentences)
    return average_sentence_length

def list_to_string(s):
    return ' '.join([str(elem) for elem in s])

In [11]:
sentences = break_sentences(list_to_string(example))

In [12]:
#Splits the text into words and count them
number_pattern = "^\\d+$"

def word_count(text):
    result = []
    sentences = break_sentences(text)
    count = 0
    for sentence in tqdm(sentences):
        words = word_tokenize(str(sentence))
        for token in words:
            token = str(token).strip(string.punctuation)
            #numbers do not make the text more difficult, they just bias the index to see numbers as easy words
            #I still want to keep words like hello45, maybe they refer to a specific race, mob or whatever in the Boardgame
            #numbers usually are inside tables or counting pages, not so much related to the readability.
            if str(token) not in string.punctuation and len(str(token)) != 0:
                if  re.match(number_pattern,str(token)) == None:
                    count += 1
    return count

In [13]:
#word_count(list_to_string(example))

In [14]:
#avg_sentence_length(list_to_string(example))

## Extract random sentences from rulebooks so I can use them in another dataset to train for example a model to determine how much luck there is in the rules of the rulebook
### (May take a while, use it if you dont have already the dataset of random sentences and you want a place to start)

In [23]:
import random

random_sentences = []
for i in tqdm(data.Name):
    s = break_sentences(list_to_string(rulebooks[i]))
    for j in range(5):
        random_s = s[random.randrange(0, len(s))]
        random_sentences.append(random_s)
        
random_sentences

100%|██████████████████████████████████████████████████████████████████████████████████| 74/74 [05:39<00:00,  4.59s/it]


[
 
  1
 X
 X
 
 
 
 
 End of round:Monster actions:
 
 
 
 On turn:Initiative [ ]:
 
 
 
 B
  B
 Bor
 AA
 AChoose one pathStart of Round:
 ǱǱReduceǱelementǱstrength.ǱǱOptionalǱshortǱrest:ǱloseǱoneǱrandomǱdiscardǱandǱrecoverǱtheǱrest.,
 
 
  Add “PUSH 2” and the earth element effects to your attack,,
  The  guard  at  the  wall  looks  at 
 you passively.,
 
  They will not move or attack unless these 
 actions are listed on their card.,
 
 
 No effect.,
 
 A unit conducting Offensive Fire may fire
 at any one enemy unit within its range,
 subject to the following hierarchy:
 1.,
 
 8.2.1  Bayonet Procedure
 1) Declaration.,
 Pool Placement Segment:  All
 remaining available LIMs are
 placed in an opaque container
 from which they can be picked
 blindly.,
 
 4.1.2  Leaders.,
 When a
 corps LIM is added, any division LIMs
 from that corps already in the pool are
 removed.,
 Each ordered unit (except 
 
 war machines) may move two hexes and still battle.,
 Each side selects an Overall Co

# Counting syllables

In [25]:
import pronouncing
p_list = pronouncing.phones_for_word("Randall")
pronouncing.syllable_count(p_list[0])

2

In [26]:
import string
import re



def syllables_count(word):
    p_list = pronouncing.phones_for_word(word)
    try:
        return pronouncing.syllable_count(p_list[0])
    except:
        return 0
     

def syllables_count_text(text):
    tot_syllables = 0
    sentences = break_sentences(text)
    for s in tqdm(sentences):
        words = word_tokenize(str(s))
        for w in words:
            if str(w) not in string.punctuation and len(str(w)) != 0:
                #a board games is not complex if it has many numbers, but if it has complex rules!
                if  re.match(number_pattern,str(w)) == None: #delete numbers because usually there are tables full of them or a the start of chapters
                    w = str(w).strip(string.punctuation)
                    #print(str(w), syllables_count(str(w)))
                    tot_syllables += syllables_count(str(w))
    return tot_syllables

def avg_syllables_word(text):
    return syllables_count_text(text) / word_count(text)

In [27]:
#syllables_count_text("Todd, thanks for connecting on LinkedIn. I have an IT opportunity that fits what you’re looking for. Would you like to set up a call? – Randall")

In [28]:
#avg_syllables_word("Todd, thanks for connecting on LinkedIn. I have an IT opportunity that fits what you’re looking for. Would you like to set up a call? – Randall")

In [29]:
# difficult words are those with syllables >= 2
# if we are counting  DaleChall, complex words are the one in not in the wordlist

In [30]:
#difficult words, not DaleChall

#Count the "complex" words consisting of three or more syllables. Do not include proper nouns,
#familiar jargon, or compound words. Do not include common suffixes (such as -es, -ed, or -ing) as a syllable;
from nltk.corpus import stopwords 
import nltk

tags_to_reduce = ['VBZ', 'VBP', 'VBN', 'VBG', 'VBD']

def difficult_words(text, threshold_syllables = 3):
     
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)
    # Find all words in the text
    count = 0
    sentences = break_sentences(text)
    for sentence in tqdm(sentences):
        s_words = word_tokenize(str(sentence))
        for w in s_words:
            if w not in string.punctuation:
                w = str(w).strip(string.punctuation)
                syllable_count = syllables_count(w)
                tag = nltk.pos_tag([w])[0][1] #take the tag, use it to not count proper nouns
                if tag in tags_to_reduce:
                    syllable_count = syllable_count - 1 # (such as -es, -ed, or -ing)
                if syllable_count >= threshold_syllables and w not in set(stopwords.words('english')) and tag != 'NNP':
                    count += 1
    return count

In [31]:
#difficult_words("Todd, thanks for connecting on LinkedIn. I have an IT opportunity that fits what you’re looking for. Would you like to set up a call? – Randall",2)

In [32]:
#difficult_words("Todd, thanks for connecting on LinkedIn. I have an IT opportunity that fits what you’re looking for. Would you like to set up a call? – Randall",3)

# Flesch reading ease

In [33]:
  """
        Implements Flesch Formula:
        Reading Ease score = 206.835 - (1.015 × ASL) - (84.6 × ASW)
        Here,
          ASL = average sentence length (number of words
                divided by number of sentences)
          ASW = average word length in syllables (number of syllables
                divided by number of words)
    """
#• Scores between 90.0 and 100.0 are considered easily understandable by an average 5th grader.
#• Scores between 60.0 and 70.0 are considered easily understood by 8th and 9th graders.
#• Scores between 0.0 and 30.0 are considered easily understood by college graduates.

def flesch_reading_ease(text):
    RE = 206.835 - float(1.015 * avg_sentence_length(text)) -\
          float(84.6 * avg_syllables_word(text))
    return round(RE,2)


In [34]:
#flesch_reading_ease(list_to_string(example))

# Gunning Fog formula

In [35]:
def gunning_fog(text, difficult_words_threshold = 3):
    per_diff_words = 100 * (difficult_words(text, difficult_words_threshold) / word_count(text))
    grade = 0.4 * (avg_sentence_length(text) + per_diff_words)
    return round(grade, 2)

In [36]:
#gunning_fog(list_to_string(example), 3)

# SMOG formula

In [37]:
#SMOG FORMULA

#calculated on a sample of 30 sentences
#SMOG grading = 3 + (polysyllable count)^1/2.

'''
0-2	4th Grade	Very easy to read
3-6	5th Grade	Very easy to read
7-12	6th Grade	Easy to read
13-20	7th Grade	Fairly easy to read
21-42	8th & 9th Grade	Conversational English
43-56	10th Grade	Fairly difficult to read
57-72	11th Grade	Fairly difficult to read
73-90	12th Grade	Fairly difficult to read
91-110	College Freshman	Difficult to read
111-132	College Sophomore	Difficult to read
133-156	College Junior	Difficult to read
157-182	College Senior	Difficult to read
183-210	College Graduate	Very difficult to read
211+	Professional	Extremely difficult to read

As a general guide, it is a good idea to aim for a reading level 
equivalent to 6th to 8th graders, as this is what the average American can most easily comprehend.
Anything higher than that would be somewhat challenging to read.
'''

'\n0-2\t4th Grade\tVery easy to read\n3-6\t5th Grade\tVery easy to read\n7-12\t6th Grade\tEasy to read\n13-20\t7th Grade\tFairly easy to read\n21-42\t8th & 9th Grade\tConversational English\n43-56\t10th Grade\tFairly difficult to read\n57-72\t11th Grade\tFairly difficult to read\n73-90\t12th Grade\tFairly difficult to read\n91-110\tCollege Freshman\tDifficult to read\n111-132\tCollege Sophomore\tDifficult to read\n133-156\tCollege Junior\tDifficult to read\n157-182\tCollege Senior\tDifficult to read\n183-210\tCollege Graduate\tVery difficult to read\n211+\tProfessional\tExtremely difficult to read\n\nAs a general guide, it is a good idea to aim for a reading level \nequivalent to 6th to 8th graders, as this is what the average American can most easily comprehend.\nAnything higher than that would be somewhat challenging to read.\n'

In [38]:
import random

#taking a sample of 30 sentences
def Smog_formula(text):
    sample = []
    i = 0
    sentences = break_sentences(text)
    count = sentence_count(text)
    while i < 30:
        sample.append(sentences[random.randrange(0, count)])
        i += 1
    hard_w = difficult_words(list_to_string(sample))
    return round(3 + (hard_w **0.5),2)

In [39]:
#Smog_formula(list_to_string(example))

# Dale Chall readability score

In [40]:
DL_list = []
my_file = open('DaleChallEasyWordList.txt', 'r')
DL_list_ten = my_file.read()
DL_list_ten = list_to_string(DL_list_ten).split("\n")

for i in DL_list_ten:
    DL_list.append(i.replace(" ", ""))

DL_list

['a',
 'able',
 'aboard',
 'about',
 'above',
 'absent',
 'accept',
 'accident',
 'account',
 'ache',
 'aching',
 'acorn',
 'acre',
 'across',
 'act',
 'acts',
 'add',
 'address',
 'admire',
 'adventure',
 'afar',
 'afraid',
 'after',
 'afternoon',
 'afterward',
 'afterwards',
 'again',
 'against',
 'age',
 'aged',
 'ago',
 'agree',
 'ah',
 'ahead',
 'aid',
 'aim',
 'air',
 'airfield',
 'airplane',
 'airport',
 'airship',
 'airy',
 'alarm',
 'alike',
 'alive',
 'all',
 'alley',
 'alligator',
 'allow',
 'almost',
 'alone',
 'along',
 'aloud',
 'already',
 'also',
 'always',
 'am',
 'America',
 'American',
 'among',
 'amount',
 'an',
 'and',
 'angel',
 'anger',
 'angry',
 'animal',
 'another',
 'answer',
 'ant',
 'any',
 'anybody',
 'anyhow',
 'anyone',
 'anything',
 'anyway',
 'anywhere',
 'apart',
 'apartment',
 'ape',
 'apiece',
 'appear',
 'apple',
 'April',
 'apron',
 'are',
 "aren't",
 'arise',
 'arithmetic',
 'arm',
 'armful',
 'army',
 'arose',
 'around',
 'arrange',
 'arrive',
 

In [41]:
def complex_words_DL(text):
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)
    # Find all words in the text
    count = 0
    sentences = break_sentences(text)
    for sentence in tqdm(sentences):
        s_words = word_tokenize(str(sentence))
        for w in s_words:
            if w not in DL_list:
                count += 1
    return count

In [42]:
#complex_words_DL(list_to_string(example))

In [43]:
#word_count(list_to_string(example))

In [44]:
def dale_chall_readability_score(text):
    """
        Implements Dale Challe Formula:
        Raw score = 0.1579*(PDW) + 0.0496*(ASL) + 3.6365
        Here,
            PDW = Percentage of difficult words.
            ASL = Average sentence length
            
4.9 or lower	easily understood by an average 4th-grade student or lower
5.0–5.9	easily understood by an average 5th or 6th-grade student
6.0–6.9	easily understood by an average 7th or 8th-grade student
7.0–7.9	easily understood by an average 9th or 10th-grade student
8.0–8.9	easily understood by an average 11th or 12th-grade student
9.0–9.9	easily understood by an average 13th to 15th-grade (college) student

    """
    words = word_count(text)
    if words > 0:
        # Percentage of words not on difficult word list
        
        #I should write here complex_words_DL(text)
        #but the word_list is too simple and keeps into consideration on basic words and basic forms 
        #so it's more usefull to use the function difficult_words that keeps the words to be hard with >= 3 syllables
        diff_words = (float( difficult_words(text)) / float(words)) * 100 
 
    raw_score = (0.1579 * diff_words) + \
                (0.0496 * avg_sentence_length(text))
     
    # If Percentage of Difficult Words is greater than 5 %, then;
    # Adjusted Score = Raw Score + 3.6365,
    # otherwise Adjusted Score = Raw Score
 
    if diff_words > 5:      
 
        raw_score += 3.6365
         
    return round(raw_score,2)

In [45]:
#dale_chall_readability_score(list_to_string(example))

# Powers-Sumner-Kear Readability Formula;

In [46]:
'''
RA = 0.0778(ASL) + 0.0455(NS) + 2.7971

Where,
RA = Reading Age
ASL = Average Sentence Length
NS = Number of Syllables

need to be computed on a passage of 100 words
'''
#passage of l words
def samplePassage(text, l):
    wc = word_count(text)
    result = []
    total_words = []
    sentences = break_sentences(text)
    for sentence in tqdm(sentences):
        words = word_tokenize(str(sentence))
        for token in words:
            token = str(token).strip(string.punctuation)
            if token not in string.punctuation and len(token) != 0:
                total_words.append(token)
    
    #random l words
    starting_index = random.randrange(0, wc-l)
    result = total_words[starting_index: starting_index+l] 
    return result

In [47]:
def PSK_readability(text):
    sample = list_to_string(samplePassage(text, 100))
    ASL = 100 /sentence_count(sample)
    NS = avg_syllables_word(sample) * 100
    RA = 0.0778*ASL + 0.0455*NS + 2.7971
    return round(RA, 2)

In [48]:
#PSK_readability(list_to_string(example))

# FORCAST readability formula

In [49]:
'''
Grade level = 20 − (N / 10)

Where N = number of single-syllable words in a 150-word sample
'''

'\nGrade level = 20 − (N / 10)\n\nWhere N = number of single-syllable words in a 150-word sample\n'

In [50]:
def Forcast_readability(text):
    sample = list_to_string(samplePassage(text, 150))
    complex_words = difficult_words(sample, 2)
    N = 150 - complex_words
    return round(20 - (N/10), 2)

In [51]:
#Forcast_readability(list_to_string(example))

# Use this part of code only for the newly added elements in the dataset, without computing all the features for all the elements

 #################################################################################################################

### make sure you have ran the code for the definition of the indixes and the code for the upload of the PDFs (this code is located from the start of the jupyter notebook until this specific cell)

In [78]:
#insert the name of the elements newly added to the dataset
newly_added = ['sumatra'] 

In [79]:
features = {}
for i in newly_added:
    feature_el = []
    feature_el.append(Forcast_readability(list_to_string(rulebooks[i])))
    feature_el.append(Smog_formula(list_to_string(rulebooks[i])))
    feature_el.append(flesch_reading_ease(list_to_string(rulebooks[i])))
    feature_el.append(PSK_readability(list_to_string(rulebooks[i])))
    feature_el.append(dale_chall_readability_score(list_to_string(rulebooks[i])))
    feature_el.append(gunning_fog(list_to_string(rulebooks[i])))
    features[i] = feature_el
  

100%|██████████████████████████████████████████████████████████████████████████████| 196/196 [00:00<00:00, 5764.68it/s]


In [80]:
for i in newly_added: #iterate on keys
    features[i].append(sentence_count(list_to_string(rulebooks[i])))

In [81]:
features

{'sumatra': [9.2, 11.06, 64.01, 11.68, 6.37, 12.1, 196]}

# Save the new computed features of the new elements so I can add them to the text file of the overall features (remember to manually copy and pase the features from the features_newlyAdded.txt of the new added elements to the features.txt of the overall features)

### Save features (only new adds)

In [82]:
file_newAdds = 'features_newlyAdded.txt'

import json
dict_f = features
with open(file_newAdds,"w") as file:
    file.write(json.dumps(dict_f))

############################################################################################################################

## Use this extraction code if you need to extract features from ALL the rulebooks 

## Extracting features, specifically extracting redability indixes (may take a while)

In [148]:
features = {}
for i in rulebooks: #iterate on keys
    feature_el = []
    feature_el.append(Forcast_readability(list_to_string(rulebooks[i])))
    feature_el.append(Smog_formula(list_to_string(rulebooks[i])))
    feature_el.append(flesch_reading_ease(list_to_string(rulebooks[i])))
    feature_el.append(PSK_readability(list_to_string(rulebooks[i])))
    feature_el.append(dale_chall_readability_score(list_to_string(rulebooks[i])))
    feature_el.append(gunning_fog(list_to_string(rulebooks[i])))
    features[i] = feature_el
    
features

100%|████████████████████████████████████████████████████████████████████████████| 2118/2118 [00:00<00:00, 6391.29it/s]


100%|████████████████████████████████████████████████████████████████████████████| 1964/1964 [00:00<00:00, 6485.71it/s]


100%|████████████████████████████████████████████████████████████████████████████| 1249/1249 [00:00<00:00, 6009.05it/s]


100%|██████████████████████████████████████████████████████████████████████████████| 988/988 [00:00<00:00, 6720.67it/s]


100%|████████████████████████████████████████████████████████████████████████████| 2053/2053 [00:00<00:00, 6558.75it/s]


100%|████████████████████████████████████████████████████████████████████████████| 1068/1068 [00:00<00:00, 6511.56it/s]


100%|████████████████████████████████████████████████████████████████████████████| 1834/1834 [00:00<00:00, 5973.90it/s]


100%|██████████████████████████████████████████████████████████████████████████████| 546/546 [00:00<00:00, 6449.34it/s]


100%|██████████████████████████████████████████████████████████████████████████████| 655/655 [00:00<00:00, 6754.01it/s]


{'Gloomhaven': [10.9, 9.63, 68.02, 13.33, 6.14, 11.14],
 'austerlitz-1805-napoleons-greatest-victory': [10.7,
  11.0,
  61.41,
  16.72,
  6.49,
  11.27],
 'commands-colors-ancients': [9.0, 10.94, 60.67, 12.29, 6.13, 12.35],
 'conquerors-alexander-great': [8.1, 9.24, 76.19, 14.33, 5.7, 9.04],
 'dominant-species-card-game': [10.0, 10.21, 67.53, 9.98, 6.14, 12.06],
 'corsairs-and-hellcats': [9.9, 7.9, 75.95, 10.5, 5.39, 7.96],
 'downtown-air-war-over-hanoi-1965-1972': [7.8,
  8.48,
  82.04,
  13.61,
  5.71,
  9.27],
 'barbarossa-kiev-rostov-1941': [9.7, 10.35, 68.04, 14.59, 5.95, 10.17],
 'empire-sun': [8.4, 10.28, 61.65, 17.68, 6.56, 12.8],
 'europe-engulfed-wwii-european-theatre-block-game': [9.6,
  9.71,
  67.51,
  11.58,
  6.06,
  12.15],
 'fading-glory': [10.0, 9.56, 68.14, 13.42, 5.93, 10.69],
 'old-school-tactical-volume-1-fighting-eastern-fron': [9.0,
  7.9,
  76.82,
  17.25,
  5.48,
  8.45],
 'formula-motor-racing': [9.2, 9.0, 79.59, 17.27, 5.4, 9.03],
 'glory-iii': [9.5, 9.63, 6

### Extracting number of sentences

In [69]:
for i in rulebooks: #iterate on keys
    features[i].append(sentence_count(list_to_string(rulebooks[i])))

KeyboardInterrupt: 

In [70]:
features

{'Gloomhaven': [10.9, 9.63, 68.02, 13.33, 6.14, 11.14, 1506],
 'austerlitz-1805-napoleons-greatest-victory': [10.7,
  11.0,
  61.41,
  16.72,
  6.49,
  11.27,
  1192],
 'commands-colors-ancients': [9.0, 10.94, 60.67, 12.29, 6.13, 12.35, 279],
 'conquerors-alexander-great': [8.1, 9.24, 76.19, 14.33, 5.7, 9.04, 1361],
 'dominant-species-card-game': [10.0, 10.21, 67.53, 9.98, 6.14, 12.06, 188],
 'corsairs-and-hellcats': [9.9, 7.9, 75.95, 10.5, 5.39, 7.96, 952],
 'downtown-air-war-over-hanoi-1965-1972': [7.8,
  8.48,
  82.04,
  13.61,
  5.71,
  9.27,
  1691],
 'barbarossa-kiev-rostov-1941': [9.7, 10.35, 68.04, 14.59, 5.95, 10.17, 2118],
 'empire-sun': [8.4, 10.28, 61.65, 17.68, 6.56, 12.8, 2298],
 'europe-engulfed-wwii-european-theatre-block-game': [9.6,
  9.71,
  67.51,
  11.58,
  6.06,
  12.15,
  1051],
 'fading-glory': [10.0, 9.56, 68.14, 13.42, 5.93, 10.69, 633],
 'old-school-tactical-volume-1-fighting-eastern-fron': [9.0,
  7.9,
  76.82,
  17.25,
  5.48,
  8.45,
  1460],
 'formula-mot

# Extracting amount of luck based on GRU model

### train the model based on a simple dataset that I created

In [212]:
import pandas as pd
data_sentences = pd.read_csv('randomness_dataset.csv')

In [213]:
data_sentences

,Sentence,Randomness
0,You will play as one of four factions \nvying ...,0
1,Each turn is split into three phases: Birdsong...,0
2,"First, roll two dice. The attacker deals hits ...",1
3,The player taking \nhits chooses which pieces ...,0
4,Draw one card plus one card per uncovered draw...,1
...,...,...
68,In games where players have the same \n \n num...,0
69,Card Adjustments: Certain event cards may modi...,1
70,5.0 The Mongols .................................,0
71,A foot missile unit may fire only at the end o...,0


#### https://huggingface.co/distilbert-base-uncased

In [214]:
from sent2vec.vectorizer import Vectorizer

vectorizer = Vectorizer()

Initializing Bert distilbert-base-uncased
Vectorization done on cuda


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [215]:
import numpy as np
from sklearn.model_selection import train_test_split
X_init = []
for i in data_sentences.Sentence:
    X_init.append(i)
    
y = data_sentences.Randomness

In [216]:
vectorizer.run(X_init)
X = vectorizer.vectors

In [217]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

### K_means

In [292]:
from sklearn.cluster import KMeans

kmeans=KMeans(n_clusters=2)
kmeans.fit(X_train)
y_pred = kmeans.predict(X_test)

In [293]:
import sklearn.metrics as sm

sm.accuracy_score(y_test, y_pred)

0.56

In [294]:
sm.precision_score(y_test, y_pred)

0.375

In [295]:
sm.recall_score(y_test, y_pred)

0.8571428571428571

### KNN

In [296]:
from sklearn.neighbors import KNeighborsClassifier

In [297]:
knn = KNeighborsClassifier(n_neighbors = 4)
model = knn.fit(X_train, y_train)

In [298]:
y_pred = knn.predict(X_test)

In [299]:
sm.accuracy_score(y_test, y_pred)

0.76

In [300]:
sm.precision_score(y_test, y_pred)

1.0

In [301]:
sm.recall_score(y_test, y_pred)

0.14285714285714285

### Decision tree

In [330]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)

DecisionTreeClassifier()

In [331]:
y_pred = dt.predict(X_test)

In [332]:
sm.accuracy_score(y_test, y_pred)

0.76

In [333]:
sm.precision_score(y_test, y_pred)

0.5555555555555556

In [334]:
sm.recall_score(y_test, y_pred)

0.7142857142857143

In [ ]:
#MI INTERESSA DI PIù LA PRECISION O IL RECALL?
#RANDOM FOREST??

In [ ]:
#LSTM??????????''

### predict

### extract features

# Save features (all elements)

In [68]:
file_f = 'features.txt'

import json
dict_f = features
with open(file_f,"w") as file:
    file.write(json.dumps(dict_f))

# Upload (all the features from all the computed elements)
### if you have computed features for new elements make sure to put them in the right .txt file so they can be used along with the other elements (manually copy and paste)

In [137]:
file_f = 'features.txt'

import json
#reading the json file
with open(file_f, "r") as file:
    file_content = file.read()
    
features = json.loads(file_content)
features

{'Gloomhaven': [10.9, 9.63, 68.02, 13.33, 6.14, 11.14, 1506],
 'austerlitz-1805-napoleons-greatest-victory': [10.7,
  11.0,
  61.41,
  16.72,
  6.49,
  11.27,
  1192],
 'commands-colors-ancients': [9.0, 10.94, 60.67, 12.29, 6.13, 12.35, 279],
 'conquerors-alexander-great': [8.1, 9.24, 76.19, 14.33, 5.7, 9.04, 1361],
 'dominant-species-card-game': [10.0, 10.21, 67.53, 9.98, 6.14, 12.06, 188],
 'corsairs-and-hellcats': [9.9, 7.9, 75.95, 10.5, 5.39, 7.96, 952],
 'downtown-air-war-over-hanoi-1965-1972': [7.8,
  8.48,
  82.04,
  13.61,
  5.71,
  9.27,
  1691],
 'barbarossa-kiev-rostov-1941': [9.7, 10.35, 68.04, 14.59, 5.95, 10.17, 2118],
 'empire-sun': [8.4, 10.28, 61.65, 17.68, 6.56, 12.8, 2298],
 'europe-engulfed-wwii-european-theatre-block-game': [9.6,
  9.71,
  67.51,
  11.58,
  6.06,
  12.15,
  1051],
 'fading-glory': [10.0, 9.56, 68.14, 13.42, 5.93, 10.69, 633],
 'old-school-tactical-volume-1-fighting-eastern-fron': [9.0,
  7.9,
  76.82,
  17.25,
  5.48,
  8.45,
  1460],
 'formula-mot

# THE FOLLOWING FEATURES ARE EXTRACTED FROM THE XML FILES AND THUS FAST TO COMPUTE

### Extract xml files

In [84]:
#Python code to illustrate parsing of XML files
# importing the required modules
import csv
import requests
import xml.etree.ElementTree as ET
  
for row in data.itertuples():
    name = row[1]
    id = row[2]    
    # url of rss feed
    url = 'https://api.geekdo.com/xmlapi/boardgame/' + str(id)
  
    # creating HTTP response object from given url
    resp = requests.get(url)
    xml_file = 'xml_files/'+name+'.xml'

    with open(xml_file, 'wb') as f:
            f.write(resp.content)
    f.close()

# Extracting average age for playingn the game

In [85]:
def avg_voted_age(xml_file, threshold_min_voting_numbers= 5):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    res = 0
    div = 0
    for item in root.findall(".//poll[@name='suggested_playerage']//result"): #xml parsing
        if item.attrib['value'] == '21 and up':
            res += 21 * int(item.attrib['numvotes'])
            div += int(item.attrib['numvotes'])
        else:
            res += int(item.attrib['value']) * int(item.attrib['numvotes'])
            div += int(item.attrib['numvotes'])
    if div > threshold_min_voting_numbers:
        return round(res/div,2)
    return root.findall(".//age")[0].text #default value

In [86]:
for name in data.Name:
    xml_file = 'xml_files/'+name+'.xml' 
    features[name].append(avg_voted_age(xml_file))
    print(features[name])

#first 6 numbers are based on the readability indexes, the last number is the avg age to play the game according to users

[10.9, 9.63, 68.02, 13.33, 6.14, 11.14, 1506, 12.74]
[10.7, 11.0, 61.41, 16.72, 6.49, 11.27, 1192, '12']
[9.0, 10.94, 60.67, 12.29, 6.13, 12.35, 279, 11.0]
[8.1, 9.24, 76.19, 14.33, 5.7, 9.04, 1361, '12']
[10.0, 10.21, 67.53, 9.98, 6.14, 12.06, 188, 8.75]
[9.9, 7.9, 75.95, 10.5, 5.39, 7.96, 952, '12']
[7.8, 8.48, 82.04, 13.61, 5.71, 9.27, 1691, '14']
[9.7, 10.35, 68.04, 14.59, 5.95, 10.17, 2118, 15.33]
[8.4, 10.28, 61.65, 17.68, 6.56, 12.8, 2298, 15.72]
[9.6, 9.71, 67.51, 11.58, 6.06, 12.15, 1051, 14.46]
[10.0, 9.56, 68.14, 13.42, 5.93, 10.69, 633, 12.75]
[9.0, 7.9, 76.82, 17.25, 5.48, 8.45, 1460, '0']
[9.2, 9.0, 79.59, 17.27, 5.4, 9.03, 116, 8.0]
[9.5, 9.63, 69.6, 17.13, 6.12, 11.19, 655, '12']
[9.1, 9.56, 70.72, 12.5, 5.83, 9.91, 1753, '12']
[8.3, 10.75, 72.61, 12.39, 6.16, 10.62, 1964, 13.0]
[9.2, 11.06, 70.1, 13.63, 6.21, 11.92, 1644, 14.89]
[9.3, 9.56, 76.35, 12.25, 5.77, 9.48, 1656, '0']
[10.4, 10.55, 74.39, 12.48, 6.06, 10.35, 2167, '12']
[9.1, 8.57, 71.95, 10.7, 5.9, 9.97, 1359

# Extratring dependecy on text when playing the game
### basically means: do you really need to read the cards and the text?

In [87]:
'''
example:
<result level="1" value="No necessary in-game text" numvotes="1"/>
<result level="2" value="Some necessary text - easily memorized or small crib sheet" numvotes="0"/>
<result level="3" value="Moderate in-game text - needs crib sheet or paste ups" numvotes="2"/>
<result level="4" value="Extensive use of text - massive conversion needed to be playable" numvotes="43"/>
<result level="5" value="Unplayable in another language" numvotes="16"/>
'''

'\nexample:\n<result level="1" value="No necessary in-game text" numvotes="1"/>\n<result level="2" value="Some necessary text - easily memorized or small crib sheet" numvotes="0"/>\n<result level="3" value="Moderate in-game text - needs crib sheet or paste ups" numvotes="2"/>\n<result level="4" value="Extensive use of text - massive conversion needed to be playable" numvotes="43"/>\n<result level="5" value="Unplayable in another language" numvotes="16"/>\n'

In [88]:
def dependency_on_language(xml_file): #from 1 to 5
    tree = ET.parse(xml_file)
    root = tree.getroot()
    res = 0
    div = 0
    for item in root.findall(".//poll[@name='language_dependence']//result"):
        res += int(item.attrib['level']) * int(item.attrib['numvotes'])
        div += int(item.attrib['numvotes'])
    
    if div > 1: #num_votes > min threshold
        return round(res/div,2) #weighted level from 1 to 5, 5 means unplayable in another language, 1 means no dependency on text
    return res #default value, in the middle of the scale.

In [89]:
for name in data.Name:
    xml_file = 'xml_files/'+name+'.xml' 
    features[name].append(dependency_on_language(xml_file))
    print(features[name])
    
#first 6 = read index, last 2 = age, dependency on text

[10.9, 9.63, 68.02, 13.33, 6.14, 11.14, 1506, 12.74, 4.18]
[10.7, 11.0, 61.41, 16.72, 6.49, 11.27, 1192, '12', 3.0]
[9.0, 10.94, 60.67, 12.29, 6.13, 12.35, 279, 11.0, 3.08]
[8.1, 9.24, 76.19, 14.33, 5.7, 9.04, 1361, '12', 3]
[10.0, 10.21, 67.53, 9.98, 6.14, 12.06, 188, 8.75, 2.12]
[9.9, 7.9, 75.95, 10.5, 5.39, 7.96, 952, '12', 2.5]
[7.8, 8.48, 82.04, 13.61, 5.71, 9.27, 1691, '14', 4.0]
[9.7, 10.35, 68.04, 14.59, 5.95, 10.17, 2118, 15.33, 3.67]
[8.4, 10.28, 61.65, 17.68, 6.56, 12.8, 2298, 15.72, 4.0]
[9.6, 9.71, 67.51, 11.58, 6.06, 12.15, 1051, 14.46, 2.69]
[10.0, 9.56, 68.14, 13.42, 5.93, 10.69, 633, 12.75, 3.22]
[9.0, 7.9, 76.82, 17.25, 5.48, 8.45, 1460, '0', 3]
[9.2, 9.0, 79.59, 17.27, 5.4, 9.03, 116, 8.0, 2.22]
[9.5, 9.63, 69.6, 17.13, 6.12, 11.19, 655, '12', 2]
[9.1, 9.56, 70.72, 12.5, 5.83, 9.91, 1753, '12', 2.8]
[8.3, 10.75, 72.61, 12.39, 6.16, 10.62, 1964, 13.0, 2.67]
[9.2, 11.06, 70.1, 13.63, 6.21, 11.92, 1644, 14.89, 3.08]
[9.3, 9.56, 76.35, 12.25, 5.77, 9.48, 1656, '0', 3.0]


# Extracting playtime (usually the longer the time the harder the game, 20 minutes play games do not have many rules!)

In [90]:
def playtime(xml_file): 
    tree = ET.parse(xml_file)
    root = tree.getroot()
    return float(root.findall(".//playingtime")[0].text)
    

In [91]:
#ADD playtime to features

for name in data.Name:
    xml_file = 'xml_files/'+name+'.xml' 
    features[name].append(playtime(xml_file))
    print(features[name])

[10.9, 9.63, 68.02, 13.33, 6.14, 11.14, 1506, 12.74, 4.18, 120.0]
[10.7, 11.0, 61.41, 16.72, 6.49, 11.27, 1192, '12', 3.0, 240.0]
[9.0, 10.94, 60.67, 12.29, 6.13, 12.35, 279, 11.0, 3.08, 60.0]
[8.1, 9.24, 76.19, 14.33, 5.7, 9.04, 1361, '12', 3, 240.0]
[10.0, 10.21, 67.53, 9.98, 6.14, 12.06, 188, 8.75, 2.12, 60.0]
[9.9, 7.9, 75.95, 10.5, 5.39, 7.96, 952, '12', 2.5, 90.0]
[7.8, 8.48, 82.04, 13.61, 5.71, 9.27, 1691, '14', 4.0, 240.0]
[9.7, 10.35, 68.04, 14.59, 5.95, 10.17, 2118, 15.33, 3.67, 240.0]
[8.4, 10.28, 61.65, 17.68, 6.56, 12.8, 2298, 15.72, 4.0, 360.0]
[9.6, 9.71, 67.51, 11.58, 6.06, 12.15, 1051, 14.46, 2.69, 720.0]
[10.0, 9.56, 68.14, 13.42, 5.93, 10.69, 633, 12.75, 3.22, 90.0]
[9.0, 7.9, 76.82, 17.25, 5.48, 8.45, 1460, '0', 3, 60.0]
[9.2, 9.0, 79.59, 17.27, 5.4, 9.03, 116, 8.0, 2.22, 20.0]
[9.5, 9.63, 69.6, 17.13, 6.12, 11.19, 655, '12', 2, 240.0]
[9.1, 9.56, 70.72, 12.5, 5.83, 9.91, 1753, '12', 2.8, 240.0]
[8.3, 10.75, 72.61, 12.39, 6.16, 10.62, 1964, 13.0, 2.67, 240.0]
[9.2, 

In [92]:
features['sumatra']

[9.2, 11.06, 64.01, 11.68, 6.37, 12.1, 196, '8', 1, 45.0]

# Creating dataset for learning activities

In [93]:
import numpy as np
from sklearn.model_selection import train_test_split
X = []
for name in data.Name:
    X.append(features[name])

In [94]:
float(data.loc[data['Name'] == 'sumatra'].weight)

1.88

In [95]:
y = data.weight

In [96]:
mat = X
        
#convert everything to float
for i in range(0,len(mat)):
    for j in range(0,len(mat[i])):
        mat[i][j] = float(mat[i][j])

In [97]:
X = np.array(X)
y = np.array(y)
X,y

(array([[1.090e+01, 9.630e+00, 6.802e+01, 1.333e+01, 6.140e+00, 1.114e+01,
         1.506e+03, 1.274e+01, 4.180e+00, 1.200e+02],
        [1.070e+01, 1.100e+01, 6.141e+01, 1.672e+01, 6.490e+00, 1.127e+01,
         1.192e+03, 1.200e+01, 3.000e+00, 2.400e+02],
        [9.000e+00, 1.094e+01, 6.067e+01, 1.229e+01, 6.130e+00, 1.235e+01,
         2.790e+02, 1.100e+01, 3.080e+00, 6.000e+01],
        [8.100e+00, 9.240e+00, 7.619e+01, 1.433e+01, 5.700e+00, 9.040e+00,
         1.361e+03, 1.200e+01, 3.000e+00, 2.400e+02],
        [1.000e+01, 1.021e+01, 6.753e+01, 9.980e+00, 6.140e+00, 1.206e+01,
         1.880e+02, 8.750e+00, 2.120e+00, 6.000e+01],
        [9.900e+00, 7.900e+00, 7.595e+01, 1.050e+01, 5.390e+00, 7.960e+00,
         9.520e+02, 1.200e+01, 2.500e+00, 9.000e+01],
        [7.800e+00, 8.480e+00, 8.204e+01, 1.361e+01, 5.710e+00, 9.270e+00,
         1.691e+03, 1.400e+01, 4.000e+00, 2.400e+02],
        [9.700e+00, 1.035e+01, 6.804e+01, 1.459e+01, 5.950e+00, 1.017e+01,
         2.118e+03, 1.

In [98]:
X.shape,y.shape

((74, 10), (74,))

# I want to predict the right weight, this is a regression task

In [99]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [100]:
X_train

array([[9.700e+00, 1.035e+01, 6.804e+01, 1.459e+01, 5.950e+00, 1.017e+01,
        2.118e+03, 1.533e+01, 3.670e+00, 2.400e+02],
       [9.900e+00, 1.068e+01, 6.764e+01, 1.231e+01, 6.060e+00, 1.038e+01,
        7.270e+02, 1.200e+01, 3.000e+00, 3.000e+02],
       [9.200e+00, 8.570e+00, 7.271e+01, 1.629e+01, 5.900e+00, 9.550e+00,
        7.580e+02, 1.200e+01, 2.000e+00, 8.400e+02],
       [1.030e+01, 9.000e+00, 7.310e+01, 1.708e+01, 5.940e+00, 9.920e+00,
        2.400e+03, 1.200e+01, 3.800e+00, 3.600e+02],
       [1.010e+01, 9.240e+00, 7.082e+01, 1.367e+01, 5.920e+00, 1.008e+01,
        1.068e+03, 1.200e+01, 3.000e+00, 3.600e+02],
       [1.010e+01, 9.710e+00, 7.075e+01, 1.149e+01, 5.970e+00, 9.820e+00,
        1.751e+03, 1.200e+01, 3.000e+00, 1.800e+02],
       [9.100e+00, 8.570e+00, 7.195e+01, 1.070e+01, 5.900e+00, 9.970e+00,
        1.359e+03, 1.200e+01, 2.000e+00, 2.400e+02],
       [9.000e+00, 1.100e+01, 6.773e+01, 1.169e+01, 6.210e+00, 1.107e+01,
        1.429e+03, 1.350e+01, 4.000e+

In [101]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [102]:
y_pred = model.predict(X_test)
y_pred

array([2.50281678, 2.80548563, 3.57680389, 3.46846328, 2.12243244,
       2.51045032, 2.9627597 , 2.89047659, 2.26586814, 3.68168269,
       2.72902795, 3.35345094, 3.44487268, 3.03352102, 2.76968368,
       3.26190887, 3.40276626, 0.33855471, 2.51868934, 3.62513286,
       3.01919342, 3.72176708, 2.98280175, 2.38842978, 2.89826151])

In [103]:
y_test

array([1.91, 2.37, 3.78, 3.88, 2.9 , 1.88, 2.58, 2.41, 1.18, 3.13, 3.25,
       3.63, 3.77, 2.78, 2.16, 3.72, 2.75, 2.52, 2.11, 3.61, 3.71, 3.86,
       2.74, 4.13, 2.85])

In [104]:
print("r2: %.3f" % model.score(X_test, y_test))
#
# Create the mean squared error
#
mse = mean_squared_error(y_test, model.predict(X_test))
print("The mean squared error (MSE) on test set: {:.4f}".format(mse))

r2: 0.052
The mean squared error (MSE) on test set: 0.5463


# Gradient boosting

In [105]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error

gbr_params = {'n_estimators': 500,
          'max_depth': 3,
          'min_samples_split': 5,
          'learning_rate': 0.01,
          'loss': 'squared_error'}
#
# Create an instance of gradient boosting regressor
#
gbr = GradientBoostingRegressor(**gbr_params)
#
# Fit the model
#
gbr.fit(X_train, y_train)

# Print Coefficient of determination R^2
print("r2: %.3f" % gbr.score(X_test, y_test))
# Create the mean squared error
mse = mean_squared_error(y_test, gbr.predict(X_test))
print("The mean squared error (MSE) on test set: {:.4f}".format(mse))

r2: 0.590
The mean squared error (MSE) on test set: 0.2359


In [106]:
#Gradient boosting performs better than L.R. but there are still many errors, the predicted results are all around 3.
#possible problems: 
#1. features are not so good
#2. I do not have enought data

# Regression tree

In [107]:
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor 
  
regressor = DecisionTreeRegressor(random_state = 0) 
  
regressor.fit(X_train, y_train)

DecisionTreeRegressor(random_state=0)

In [108]:
y_pred = regressor.predict(X_test)

In [109]:
# Print Coefficient of determination R^2
print("r2: %.3f" % regressor.score(X_test, y_test))
# Create the mean squared error
mse = mean_squared_error(y_test, regressor.predict(X_test))
print("The mean squared error (MSE) on test set: {:.4f}".format(mse))

r2: 0.529
The mean squared error (MSE) on test set: 0.2713


In [110]:
y_pred

array([2.69, 2.76, 3.39, 2.74, 2.76, 2.22, 2.74, 2.02, 1.61, 3.74, 2.02,
       3.61, 3.32, 3.61, 2.02, 3.61, 3.06, 2.48, 2.81, 3.37, 3.43, 4.24,
       3.31, 3.91, 2.7 ])

In [111]:
y_test

array([1.91, 2.37, 3.78, 3.88, 2.9 , 1.88, 2.58, 2.41, 1.18, 3.13, 3.25,
       3.63, 3.77, 2.78, 2.16, 3.72, 2.75, 2.52, 2.11, 3.61, 3.71, 3.86,
       2.74, 4.13, 2.85])

In [112]:
X_train_less_features = []
X_test_less_features = []
for i in X_train:
    X_train_less_features.append(i[5:])
    
for i in X_test:
    X_test_less_features.append(i[5:])

In [113]:
regressor = DecisionTreeRegressor(random_state = 0) 
  
regressor.fit(X_train_less_features, y_train)

DecisionTreeRegressor(random_state=0)

In [114]:
y_pred = regressor.predict(X_test_less_features)

In [115]:
# Print Coefficient of determination R^2
print("r2:  %.3f" % regressor.score(X_test_less_features, y_test))
# Create the mean squared error
mse = mean_squared_error(y_test, regressor.predict(X_test_less_features))
print("The mean squared error (MSE) on test set: {:.4f}".format(mse))

r2:  0.355
The mean squared error (MSE) on test set: 0.3713


In [116]:
gbr_params = {'n_estimators': 500,
          'max_depth': 3,
          'min_samples_split': 5,
          'learning_rate': 0.01,
          'loss': 'squared_error'}
#
# Create an instance of gradient boosting regressor
#
gbr = GradientBoostingRegressor(**gbr_params)
#
# Fit the model
#
gbr.fit(X_train_less_features, y_train)

# Print Coefficient of determination R^2
print("r2: %.3f" % gbr.score(X_test_less_features, y_test))
# Create the mean squared error
mse = mean_squared_error(y_test, gbr.predict(X_test_less_features))
print("The mean squared error (MSE) on test set: {:.4f}".format(mse))

r2: 0.569
The mean squared error (MSE) on test set: 0.2480


In [117]:
gbr.predict(X_test_less_features)

array([2.17389316, 2.24644031, 3.28600178, 3.46688089, 2.34230893,
       1.83371651, 3.4710327 , 3.10302743, 1.79521164, 3.7016796 ,
       3.31197342, 3.33395786, 3.1627584 , 3.13288385, 3.07026549,
       3.15012   , 3.3139083 , 2.41747382, 2.37558947, 3.4501569 ,
       3.4710327 , 3.88950062, 3.29592878, 3.4229721 , 3.46536881])

In [118]:
y_test

array([1.91, 2.37, 3.78, 3.88, 2.9 , 1.88, 2.58, 2.41, 1.18, 3.13, 3.25,
       3.63, 3.77, 2.78, 2.16, 3.72, 2.75, 2.52, 2.11, 3.61, 3.71, 3.86,
       2.74, 4.13, 2.85])

In [119]:
#I risultati sono migliori!!

# I need to understand which is the beast feature/features to use 

#### PCA

In [120]:
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(X)

In [121]:
X_train, X_test, y_train, y_test = train_test_split(principalComponents, y, test_size=0.33, random_state=42)

In [122]:
gbr_params = {'n_estimators': 500,
          'max_depth': 3,
          'min_samples_split': 5,
          'learning_rate': 0.01,
          'loss': 'squared_error'}
#
# Create an instance of gradient boosting regressor
#
gbr = GradientBoostingRegressor(**gbr_params)
#
# Fit the model
#
gbr.fit(X_train, y_train)

# Print Coefficient of determination R^2
print("Model Accuracy: %.3f" % gbr.score(X_test, y_test))
# Create the mean squared error
mse = mean_squared_error(y_test, gbr.predict(X_test))
print("The mean squared error (MSE) on test set: {:.4f}".format(mse))

Model Accuracy: 0.420
The mean squared error (MSE) on test set: 0.3340


## since Gradient-boosting seems the best model to get, try all the possible combinations of features on GB

In [123]:
#TRY JUST WITH SMOG
#X = array of features
def try_all_features(X, y, features_to_select):
    if( max(features_to_select) > len(X[0])-1):
        print("error, the vector X only has: "+ str(len(X[0])-1) + "features")
        return
    
    X_try = []
    for i in X:
        el = []
        for j in features_to_select:
            el.append(i[j])
        X_try.append(el)
    
    X_try = np.array(X_try)
    try:
        X_try.shape[1]
    except:
        X_try = X_try.reshape(-1, 1)

    X_train, X_test, y_train, y_test = train_test_split(X_try, y, test_size=0.33, random_state=42)

    #print(X_try)
    
    #Gradient boosting, #400-500 estimators
    gbr_params = {'n_estimators': 400, #400
          'max_depth': 8, #8
          'min_samples_split': 2, #2
          'learning_rate': 0.01, #0.01
          'loss': 'squared_error'}


    # Create an instance of gradient boosting regressor
    gbr = GradientBoostingRegressor(**gbr_params)

    # Fit the model
    gbr.fit(X_train, y_train)
    
    print("Features selected:", features_to_select)
    # Print Coefficient of determination R^2
    r2 = gbr.score(X_test, y_test)
    print("R^2 on test set: %.3f" % r2)
    # Create the mean squared error
    mse = mean_squared_error(y_test, gbr.predict(X_test))
    print("The mean squared error (MSE) on test set: {:.4f}".format(mse))
    print("\n")
    return r2, mse, gbr.predict(X_test), gbr

In [124]:
try_all_features(X, y, (0,1,2,3,4,5,6,7,8))

Features selected: (0, 1, 2, 3, 4, 5, 6, 7, 8)
R^2 on test set: -0.089
The mean squared error (MSE) on test set: 0.6273




(-0.0890190743976016,
 0.6273279567407972,
 array([3.60250839, 2.92489876, 3.33488274, 2.84672517, 2.9568864 ,
        3.60250839, 2.72215783, 2.96417625, 2.36110314, 3.69226047,
        2.96417625, 3.25393974, 3.26209475, 3.45871268, 3.21535081,
        3.17686179, 3.54358542, 2.44759039, 3.60250839, 3.30162671,
        3.36659761, 3.96531982, 3.45088733, 3.60250839, 2.72606099]),
 GradientBoostingRegressor(learning_rate=0.01, max_depth=8, n_estimators=400))

# try on all possible features combinatios
#### Take into consideration time, if we have lots of features this process is unfeasible, in that case you'll need to select what you think are the best features and try them against the dataset and see which combination obtain the best error.
#### In this case all the combinations are bruteforced!

In [125]:
from itertools import combinations
features_to_select = [i for i in range(len(X[0]))]
list_combinations = list()
for n in range(len(features_to_select) + 1):
    list_combinations += list(combinations(features_to_select, n))

In [126]:
max_r2 = 0
max_r2_combination = ()
min_mse = 1
min_mse_combination = ()
best_r2_predictions = []
best_mse_predictions = []
best_model_mse = GradientBoostingRegressor()
best_model_r2 = GradientBoostingRegressor()
for combination in list_combinations[1:]:
    r2, mse, predictions, model= try_all_features(X, y, combination)
    if r2 > max_r2:
        max_r2 = r2
        max_r2_combination = combination
        best_r2_predictions = predictions
        best_model_mse = model
    if mse < min_mse:
        min_mse = mse
        min_mse_combination = combination
        best_mse_predictions = predictions
        best_model_r2 = model

Features selected: (0,)
R^2 on test set: -0.604
The mean squared error (MSE) on test set: 0.9237


Features selected: (1,)
R^2 on test set: -1.087
The mean squared error (MSE) on test set: 1.2024


Features selected: (2,)
R^2 on test set: -0.491
The mean squared error (MSE) on test set: 0.8590


Features selected: (3,)
R^2 on test set: -0.541
The mean squared error (MSE) on test set: 0.8877


Features selected: (4,)
R^2 on test set: -0.178
The mean squared error (MSE) on test set: 0.6788


Features selected: (5,)
R^2 on test set: -0.221
The mean squared error (MSE) on test set: 0.7033


Features selected: (6,)
R^2 on test set: 0.096
The mean squared error (MSE) on test set: 0.5205


Features selected: (7,)
R^2 on test set: 0.190
The mean squared error (MSE) on test set: 0.4667


Features selected: (8,)
R^2 on test set: -0.099
The mean squared error (MSE) on test set: 0.6331


Features selected: (9,)
R^2 on test set: 0.359
The mean squared error (MSE) on test set: 0.3692


Features sele

Features selected: (0, 5, 6)
R^2 on test set: 0.190
The mean squared error (MSE) on test set: 0.4665


Features selected: (0, 5, 7)
R^2 on test set: 0.230
The mean squared error (MSE) on test set: 0.4436


Features selected: (0, 5, 8)
R^2 on test set: -0.186
The mean squared error (MSE) on test set: 0.6829


Features selected: (0, 5, 9)
R^2 on test set: 0.215
The mean squared error (MSE) on test set: 0.4519


Features selected: (0, 6, 7)
R^2 on test set: -0.494
The mean squared error (MSE) on test set: 0.8606


Features selected: (0, 6, 8)
R^2 on test set: 0.272
The mean squared error (MSE) on test set: 0.4194


Features selected: (0, 6, 9)
R^2 on test set: 0.325
The mean squared error (MSE) on test set: 0.3889


Features selected: (0, 7, 8)
R^2 on test set: -0.223
The mean squared error (MSE) on test set: 0.7043


Features selected: (0, 7, 9)
R^2 on test set: 0.470
The mean squared error (MSE) on test set: 0.3051


Features selected: (0, 8, 9)
R^2 on test set: 0.390
The mean squared e

Features selected: (4, 7, 9)
R^2 on test set: 0.510
The mean squared error (MSE) on test set: 0.2823


Features selected: (4, 8, 9)
R^2 on test set: 0.160
The mean squared error (MSE) on test set: 0.4838


Features selected: (5, 6, 7)
R^2 on test set: 0.280
The mean squared error (MSE) on test set: 0.4147


Features selected: (5, 6, 8)
R^2 on test set: 0.129
The mean squared error (MSE) on test set: 0.5017


Features selected: (5, 6, 9)
R^2 on test set: 0.382
The mean squared error (MSE) on test set: 0.3559


Features selected: (5, 7, 8)
R^2 on test set: 0.309
The mean squared error (MSE) on test set: 0.3981


Features selected: (5, 7, 9)
R^2 on test set: 0.577
The mean squared error (MSE) on test set: 0.2438


Features selected: (5, 8, 9)
R^2 on test set: 0.084
The mean squared error (MSE) on test set: 0.5278


Features selected: (6, 7, 8)
R^2 on test set: 0.364
The mean squared error (MSE) on test set: 0.3663


Features selected: (6, 7, 9)
R^2 on test set: 0.424
The mean squared erro

Features selected: (0, 4, 6, 7)
R^2 on test set: 0.003
The mean squared error (MSE) on test set: 0.5746


Features selected: (0, 4, 6, 8)
R^2 on test set: 0.085
The mean squared error (MSE) on test set: 0.5273


Features selected: (0, 4, 6, 9)
R^2 on test set: 0.559
The mean squared error (MSE) on test set: 0.2543


Features selected: (0, 4, 7, 8)
R^2 on test set: 0.162
The mean squared error (MSE) on test set: 0.4827


Features selected: (0, 4, 7, 9)
R^2 on test set: 0.554
The mean squared error (MSE) on test set: 0.2571


Features selected: (0, 4, 8, 9)
R^2 on test set: 0.390
The mean squared error (MSE) on test set: 0.3514


Features selected: (0, 5, 6, 7)
R^2 on test set: 0.032
The mean squared error (MSE) on test set: 0.5576


Features selected: (0, 5, 6, 8)
R^2 on test set: 0.220
The mean squared error (MSE) on test set: 0.4492


Features selected: (0, 5, 6, 9)
R^2 on test set: 0.382
The mean squared error (MSE) on test set: 0.3563


Features selected: (0, 5, 7, 8)
R^2 on test se

Features selected: (2, 3, 5, 7)
R^2 on test set: -0.462
The mean squared error (MSE) on test set: 0.8424


Features selected: (2, 3, 5, 8)
R^2 on test set: -0.472
The mean squared error (MSE) on test set: 0.8482


Features selected: (2, 3, 5, 9)
R^2 on test set: 0.527
The mean squared error (MSE) on test set: 0.2722


Features selected: (2, 3, 6, 7)
R^2 on test set: -0.262
The mean squared error (MSE) on test set: 0.7270


Features selected: (2, 3, 6, 8)
R^2 on test set: 0.374
The mean squared error (MSE) on test set: 0.3603


Features selected: (2, 3, 6, 9)
R^2 on test set: 0.440
The mean squared error (MSE) on test set: 0.3225


Features selected: (2, 3, 7, 8)
R^2 on test set: -0.340
The mean squared error (MSE) on test set: 0.7716


Features selected: (2, 3, 7, 9)
R^2 on test set: 0.359
The mean squared error (MSE) on test set: 0.3690


Features selected: (2, 3, 8, 9)
R^2 on test set: 0.179
The mean squared error (MSE) on test set: 0.4730


Features selected: (2, 4, 5, 6)
R^2 on tes

...

Features selected: (0, 1, 4, 6, 7, 9)
R^2 on test set: 0.580
The mean squared error (MSE) on test set: 0.2418


Features selected: (0, 1, 4, 6, 8, 9)
R^2 on test set: 0.441
The mean squared error (MSE) on test set: 0.3218


Features selected: (0, 1, 4, 7, 8, 9)
R^2 on test set: 0.520
The mean squared error (MSE) on test set: 0.2767


Features selected: (0, 1, 5, 6, 7, 8)
R^2 on test set: -0.309
The mean squared error (MSE) on test set: 0.7539


Features selected: (0, 1, 5, 6, 7, 9)
R^2 on test set: 0.441
The mean squared error (MSE) on test set: 0.3222


Features selected: (0, 1, 5, 6, 8, 9)
R^2 on test set: 0.300
The mean squared error (MSE) on test set: 0.4031


Features selected: (0, 1, 5, 7, 8, 9)
R^2 on test set: 0.417
The mean squared error (MSE) on test set: 0.3361


Features selected: (0, 1, 6, 7, 8, 9)
R^2 on test set: 0.435
The mean squared error (MSE) on test set: 0.3253


Features selected: (0, 2, 3, 4, 5, 6)
R^2 on test set: 0.335
The mean squared error (MSE) on test set: 

Features selected: (1, 2, 3, 5, 6, 7)
R^2 on test set: -0.076
The mean squared error (MSE) on test set: 0.6199


Features selected: (1, 2, 3, 5, 6, 8)
R^2 on test set: 0.220
The mean squared error (MSE) on test set: 0.4495


Features selected: (1, 2, 3, 5, 6, 9)
R^2 on test set: 0.458
The mean squared error (MSE) on test set: 0.3121


Features selected: (1, 2, 3, 5, 7, 8)
R^2 on test set: -0.298
The mean squared error (MSE) on test set: 0.7476


Features selected: (1, 2, 3, 5, 7, 9)
R^2 on test set: 0.282
The mean squared error (MSE) on test set: 0.4136


Features selected: (1, 2, 3, 5, 8, 9)
R^2 on test set: 0.455
The mean squared error (MSE) on test set: 0.3142


Features selected: (1, 2, 3, 6, 7, 8)
R^2 on test set: -0.145
The mean squared error (MSE) on test set: 0.6595


Features selected: (1, 2, 3, 6, 7, 9)
R^2 on test set: 0.516
The mean squared error (MSE) on test set: 0.2789


Features selected: (1, 2, 3, 6, 8, 9)
R^2 on test set: 0.480
The mean squared error (MSE) on test set

Features selected: (0, 1, 2, 3, 4, 5, 8)
R^2 on test set: -0.367
The mean squared error (MSE) on test set: 0.7875


Features selected: (0, 1, 2, 3, 4, 5, 9)
R^2 on test set: 0.331
The mean squared error (MSE) on test set: 0.3855


Features selected: (0, 1, 2, 3, 4, 6, 7)
R^2 on test set: -0.113
The mean squared error (MSE) on test set: 0.6414


Features selected: (0, 1, 2, 3, 4, 6, 8)
R^2 on test set: 0.375
The mean squared error (MSE) on test set: 0.3602


Features selected: (0, 1, 2, 3, 4, 6, 9)
R^2 on test set: 0.546
The mean squared error (MSE) on test set: 0.2616


Features selected: (0, 1, 2, 3, 4, 7, 8)
R^2 on test set: -0.395
The mean squared error (MSE) on test set: 0.8035


Features selected: (0, 1, 2, 3, 4, 7, 9)
R^2 on test set: 0.390
The mean squared error (MSE) on test set: 0.3514


Features selected: (0, 1, 2, 3, 4, 8, 9)
R^2 on test set: 0.272
The mean squared error (MSE) on test set: 0.4195


Features selected: (0, 1, 2, 3, 5, 6, 7)
R^2 on test set: -0.076
The mean squ

Features selected: (0, 2, 4, 5, 7, 8, 9)
R^2 on test set: 0.593
The mean squared error (MSE) on test set: 0.2344


Features selected: (0, 2, 4, 6, 7, 8, 9)
R^2 on test set: 0.544
The mean squared error (MSE) on test set: 0.2625


Features selected: (0, 2, 5, 6, 7, 8, 9)
R^2 on test set: 0.462
The mean squared error (MSE) on test set: 0.3099


Features selected: (0, 3, 4, 5, 6, 7, 8)
R^2 on test set: 0.323
The mean squared error (MSE) on test set: 0.3899


Features selected: (0, 3, 4, 5, 6, 7, 9)
R^2 on test set: 0.560
The mean squared error (MSE) on test set: 0.2537


Features selected: (0, 3, 4, 5, 6, 8, 9)
R^2 on test set: 0.502
The mean squared error (MSE) on test set: 0.2871


Features selected: (0, 3, 4, 5, 7, 8, 9)
R^2 on test set: 0.384
The mean squared error (MSE) on test set: 0.3546


Features selected: (0, 3, 4, 6, 7, 8, 9)
R^2 on test set: 0.563
The mean squared error (MSE) on test set: 0.2519


Features selected: (0, 3, 5, 6, 7, 8, 9)
R^2 on test set: 0.503
The mean squared

Features selected: (0, 1, 4, 5, 6, 7, 8, 9)
R^2 on test set: 0.584
The mean squared error (MSE) on test set: 0.2397


Features selected: (0, 2, 3, 4, 5, 6, 7, 8)
R^2 on test set: -0.166
The mean squared error (MSE) on test set: 0.6718


Features selected: (0, 2, 3, 4, 5, 6, 7, 9)
R^2 on test set: 0.549
The mean squared error (MSE) on test set: 0.2595


Features selected: (0, 2, 3, 4, 5, 6, 8, 9)
R^2 on test set: 0.504
The mean squared error (MSE) on test set: 0.2859


Features selected: (0, 2, 3, 4, 5, 7, 8, 9)
R^2 on test set: 0.451
The mean squared error (MSE) on test set: 0.3161


Features selected: (0, 2, 3, 4, 6, 7, 8, 9)
R^2 on test set: 0.567
The mean squared error (MSE) on test set: 0.2493


Features selected: (0, 2, 3, 5, 6, 7, 8, 9)
R^2 on test set: 0.521
The mean squared error (MSE) on test set: 0.2759


Features selected: (0, 2, 4, 5, 6, 7, 8, 9)
R^2 on test set: 0.557
The mean squared error (MSE) on test set: 0.2550


Features selected: (0, 3, 4, 5, 6, 7, 8, 9)
R^2 on test

# Best features combination

In [127]:
min_mse_combination, max_r2_combination

((0, 1, 2, 4, 7, 8, 9), (0, 1, 2, 4, 7, 8, 9))

In [128]:
min_mse, max_r2 #(0.21789440539808821, 0.6217430434379843)

(0.2189006911043025, 0.6199961671564704)

In [129]:
best_mse_predictions

array([2.59643985, 2.85833752, 3.45519284, 3.82484465, 2.86881794,
       1.97401151, 3.48549116, 2.04104977, 1.577237  , 3.62022534,
       2.81886622, 3.50509588, 3.31316855, 2.83968685, 2.04104977,
       2.84121421, 3.26727748, 2.65547949, 2.60570652, 2.85307886,
       3.48352338, 3.34583568, 3.02666133, 3.49976019, 3.28654078])

In [130]:
y_test

array([1.91, 2.37, 3.78, 3.88, 2.9 , 1.88, 2.58, 2.41, 1.18, 3.13, 3.25,
       3.63, 3.77, 2.78, 2.16, 3.72, 2.75, 2.52, 2.11, 3.61, 3.71, 3.86,
       2.74, 4.13, 2.85])

In [131]:
#TO PERFORM AFTER THE BEST FEATURES WERE FOUND
def predict_one_mse(name):
    element = np.array(features[name])
    res = []
    for i in min_mse_combination:
        res.append(element[i])
    res = np.array(res)
    #prediction and real weight
    return best_model_mse.predict(res.reshape(1,-1)), float(data.loc[data['Name'] == name].weight)

In [136]:
predict_one_mse('sumatra')
#prediction vs real value

(array([1.97401151]), 1.88)

In [133]:
data

,Name,Id,ruleBook,weight
0,Gloomhaven,174430,gloomhaven-rulebook.pdf,3.88
1,austerlitz-1805-napoleons-greatest-victory,8571,aus_rules.pdf,3.20
2,commands-colors-ancients,14105,CC-EPIC_Rules.pdf,2.69
3,conquerors-alexander-great,13224,conq_alex_Rules.pdf,2.84
4,dominant-species-card-game,96260,DSC_Rules-web_FINAL.pdf,1.91
...,...,...,...,...
69,marvel-champions-card-game,285774,c2-marvel-champions-the-card-game-rules-refere...,2.89
70,ark-nova,342942,Ark-Nova-Rulebook.pdf,3.71
71,brass-birmingham,224517,Brass-Birmingham-Rulebook-2018.11.20-highlight...,3.91
72,7-wonders-duel,173346,6b-7-wonders-duel-rules.pdf,2.22
